In [ ]:
!pip install -U bitsandbytes accelerate
!pip install torch transformers llmlingua
!pip install datasets==2.21.0 huggingface-hub==0.34.0
!git clone https://github.com/ilatims-b/LLMLingua.git

In [ ]:
from datasets import load_dataset
import json
import os

In [ ]:
#if dataset is longbench

# Initialize data list
data = []

# Set the number of samples to load from each dataset(for trial)
NUM_SAMPLES = 1 
#not including chineses, coding, answer classification, passage count dataset for now

#datasets = ['narrativeqa', 'qasper', 'multifieldqa_en', 'hotpotqa', '2wikimqa', 'musique', 'gov_report', 'qmsum', 'multi_news', 'triviaqa', 'samsum', 'passage_retrieval_en']
#for dry run only doing one, my api keys go boom
datasets=['multifieldqa_en']
for dataset_ in datasets:
    data_ = load_dataset('THUDM/LongBench', dataset_, split='test')
    

    for idx, instance in enumerate(data_):
        if idx >= NUM_SAMPLES:
            break
            
        temp = {}
        temp["dataset"] = dataset_
        temp["idx"] = idx
        temp["prompt"] = instance['context'] + '\n' + instance['input']  # Added newline between context and input
        temp["answer"] = instance['answers']
        temp["length"] = instance['length']
        data.append(temp)

os.makedirs("/kaggle/working/dataset", exist_ok=True)
with open("/kaggle/working/dataset/longbench_1.json", "w") as f:
    json.dump(data, f, indent=4)
#for trial i took first sample from nultifiedqna, it has only 813 tokens nearly

In [ ]:

os.chdir('/kaggle/working/LLMLingua/experiments/llmlingua2/data_collection')

In [ ]:
!pip install openai==0.28

In [ ]:
#wanted to test from here but doesnt work because i had made openai account more than 3 months back, altho i did not use free trial credits, the acc is old so it does not allow
#also they use azure, and did not have api key parge args, added that
#generating compressed prompts from gpt4 for data annotation
!python compress.py --load_origin_from /kaggle/working/dataset/longbench_1.json \
--chunk_size 512 \
--compressor gpt4 \
--model_name gpt-4o-mini \
--api_key "yourapikey"\
--save_path /kaggle/working/dataset/longbench_1_compressed.json

In [ ]:
#train data annotations
!python label_word.py \
--load_prompt_from /kaggle/working/dataset/longbench_1_compressed.json \
--window_size 400 \
--save_path /kaggle/working/dataset/longbench_1_labelled.json \

In [ ]:
#train data filtering, they have defined various metrics in paper
!python filter.py \
--load_path /kaggle/working/dataset/longbench_1_labelled.pt\
--save_path /kaggle/working/dataset/longbench_1_filtered.pt

In [ ]:
#i have added quantization, int4, int8, float16. but for training float16 is the best we should go for. 8bit and 4bit will give errors in optimizer and loss propagation
#training on token classification using AutoModelForTokenClassification
#xlm-roberta is not gated, but if we choose to work with another model, which is gated, add hugging face login
!python train_roberta.py \
    --data_path /kaggle/input/your-dataset/train.pt \
    --eval_data_path /kaggle/input/your-dataset/val.pt \
    --save_path /kaggle/working/roberta_custom \
    --epochs 3 \
    --lr 3e-5 \
    --quantization float16 \
    --batch_size 16
